In [2]:
import os
import openai

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

### Download an example reference data point from LangSmith

In [3]:
client = Client()

In [4]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [5]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('2d9ee68f-b798-49a1-a5d4-9f475f685dd4'), created_at=datetime.datetime(2025, 10, 10, 14, 27, 58, 18781, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 10, 14, 27, 58, 18781, tzinfo=datetime.timezone.utc), example_count=63, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-15.6-arm64-arm-64bit', 'sdk_version': '0.4.33', 'runtime_version': '3.12.11', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': '0.3.78'}})

In [6]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Can you suggest any food processors or kitchen appliances you have?'}

In [8]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': 'None of the products contain a portable air conditioner feature.',
 'reference_context_ids': [],
 'reference_descriptions': []}

In [8]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[5].inputs

{'question': 'Do you have any surveillance cameras or security products?'}

In [7]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[5].outputs

{'ground_truth': 'Yes, we stock a wireless WiFi light bulb security camera with motion detection and night vision, and a hidden camera clock with HD 1080P WiFi and night vision for home or office security.',
 'reference_context_ids': ['B0C4DBSWGW', 'B0B2JJJFCD'],
 'reference_descriptions': ['XVIM Wireless WiFi Light Bulb Camera, 3MP Light Bulb Security Camera, 360° Pan/Tilt Indoor Dome Camera with LED Light, Motion Detection, Full Color Night Vision 【Wireless Connection & Easy Installation】- XVIM wireless security camera supports 2.4Ghz WiFi wireless connection (5Ghz WiFi is not supported). Bulb camera can be directly used with E27 light socket (100V~240V). Also, equipped with an adapter, the light bulb security camera can be easily used by being screwed into the adapter and plugged into the power supply. You can easily use the bulb camera even if there is no light socket at home.【3MP HD Image & Color Night Vision】- With 3MP HD resolution, XVIM light bulb security camera shows you clea

In [9]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=10))[5].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=10))[5].outputs

### RAG Pipeline

In [10]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [11]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }

def process_context(context):

    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt
    
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [12]:
rag_pipeline("Can I get some charger?", qdrant_client, top_k=5)

{'answer': 'Yes, there are several charger cables available:\n\n1. iPhone Charger Cord Lightning Cables, Original 2022 Upgraded, 3-pack of 3ft cables, Apple MFi Certified, suitable for many iPhone and iPad models (Black) - product B0BYYLJRHT.\n\n2. MUXA 6 Pack of colorful nylon Lightning cables with various lengths (3ft, 6ft, 10ft), Apple MFi Certified, compatible with many iPhone and iPad models - product B09TNXY54Y.\n\n3. 5 in 1 USB C to Multi Charging Cable (10ft), Apple MFi Certified, with Lightning, Type C, and Micro USB connectors for multiple device charging - product B0BFPZGYLD.\n\n4. GREPHONE 2 Pack USB C to Lightning Cable, 6ft, Apple MFi Certified, fast charging compatible with many iPhone and iPad models - product B0BV6PWVCG.\n\n5. Mixblu Charger Cable Replacement for Fitbit Inspire 3 (2 Pack, 3.3ft) - product B0BGDQLZD2.\n\nPlease let me know if you want details or to choose one.',
 'question': 'Can I get some charger?',
 'retrieved_context_ids': ['B0BYYLJRHT',
  'B09TNXY5

### RAGAS Metrics

In [13]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy


ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/var/folders/hd/mwwk_4zs7blcj6tc0ycm9t5c0000gn/T/ipykernel_27041/3539187904.py:5: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/hd/mwwk_4zs7blcj6tc0ycm9t5c0000gn/T/ipykernel_27041/3539187904.py:6: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [14]:
reference_input

{'question': 'Do you have any surveillance cameras or security products?'}

In [15]:
reference_output

{'ground_truth': 'Yes, we stock a wireless WiFi light bulb security camera with motion detection and night vision, and a hidden camera clock with HD 1080P WiFi and night vision for home or office security.',
 'reference_context_ids': ['B0C4DBSWGW', 'B0B2JJJFCD'],
 'reference_descriptions': ['XVIM Wireless WiFi Light Bulb Camera, 3MP Light Bulb Security Camera, 360° Pan/Tilt Indoor Dome Camera with LED Light, Motion Detection, Full Color Night Vision 【Wireless Connection & Easy Installation】- XVIM wireless security camera supports 2.4Ghz WiFi wireless connection (5Ghz WiFi is not supported). Bulb camera can be directly used with E27 light socket (100V~240V). Also, equipped with an adapter, the light bulb security camera can be easily used by being screwed into the adapter and plugged into the power supply. You can easily use the bulb camera even if there is no light socket at home.【3MP HD Image & Color Night Vision】- With 3MP HD resolution, XVIM light bulb security camera shows you clea

In [16]:
result = rag_pipeline(reference_input["question"], qdrant_client)

In [17]:
result

{'answer': 'Yes, we have the following surveillance cameras and security products available:\n\n1. XVIM Wireless WiFi Light Bulb Camera (3MP) - A 360° pan/tilt indoor dome camera with LED light, motion detection, full color night vision, two-way audio, remote access, and smart storage options. It connects via 2.4GHz WiFi and can be installed easily in an E27 light socket or with an adapter.\n\n2. Acrisbrat Hidden Camera Clock - A HD 1080P WiFi spy camera alarm clock with night vision and motion detection, suitable for home or office security. It operates via the "Lookcam" app for Android and iPhone.\n\nThese products provide options for home or office security monitoring.',
 'question': 'Do you have any surveillance cameras or security products?',
 'retrieved_context_ids': ['B0C4DBSWGW',
  'B0B2JJJFCD',
  'B0BR8Q2PXN',
  'B0CH6P8DYF',
  'B0BGLRMPQD'],
 'retrieved_context': ['XVIM Wireless WiFi Light Bulb Camera, 3MP Light Bulb Security Camera, 360° Pan/Tilt Indoor Dome Camera with LED 

In [36]:
async def ragas_faithfulness(run):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [69]:
await ragas_faithfulness(result)

1.0

In [ ]:
async def ragas_responce_relevancy(run):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [70]:
await ragas_responce_relevancy(result)

np.float64(0.9354046818505207)

In [18]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [19]:
result

{'answer': 'Yes, we have the following surveillance cameras and security products available:\n\n1. XVIM Wireless WiFi Light Bulb Camera (3MP) - A 360° pan/tilt indoor dome camera with LED light, motion detection, full color night vision, two-way audio, remote access, and smart storage options. It connects via 2.4GHz WiFi and can be installed easily in an E27 light socket or with an adapter.\n\n2. Acrisbrat Hidden Camera Clock - A HD 1080P WiFi spy camera alarm clock with night vision and motion detection, suitable for home or office security. It operates via the "Lookcam" app for Android and iPhone.\n\nThese products provide options for home or office security monitoring.',
 'question': 'Do you have any surveillance cameras or security products?',
 'retrieved_context_ids': ['B0C4DBSWGW',
  'B0B2JJJFCD',
  'B0BR8Q2PXN',
  'B0CH6P8DYF',
  'B0BGLRMPQD'],
 'retrieved_context': ['XVIM Wireless WiFi Light Bulb Camera, 3MP Light Bulb Security Camera, 360° Pan/Tilt Indoor Dome Camera with LED 

In [54]:
reference_output

{'ground_truth': 'The package includes a printing top case, a bottom case, a screen protector, and a keyboard cover.',
 'reference_context_ids': ['B0B6V8THVX'],
 'reference_descriptions': ['May Chen Compatible with Newest MacBook Air 13.6 Inch Case 2022 Release Model A2681 M2 Clip, Plastic Hard Shell Case Cover for MacBook Air 13 inch with Retina Display & Touch ID, Astronaut Flight 【 COMPATIBLE WITH 】2022 Newest MacBook Air 13.6 inch Model: A2681 Apple M2 Clip Liquid Retina display & Touch ID - Please kindly check model number "A2xxx"on the back of the for MacBook before your purchase.【 WARNING Not compatible with 】Not fit for 2018 2019 2020 2021 MacBook Air 13 inch Model A2337 M1 / A2179 / A1932 with Retina Dispaly and Touch ID. Not compatible with older Air 13 models,Please double check your model number.【 May Chen Design 】Unique 3D UV Painting,to add something different and colorful to your Macbook. Two pieces snap on design, easy to install and take off. It allows full access to a

In [20]:
await ragas_context_precision_id_based(result, reference_output)

0.4

In [22]:

async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [23]:
await ragas_context_recall_id_based(result, reference_output)

1.0